In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
  loader = DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)

  documents = loader.load()
  return documents

In [ ]:
extracted_data = load_pdf_file()  #have to give path of file

In [ ]:
#After load files now split it into chunks
def split_text(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                 chunk_overlap=20)
  text_chunks = text_splitter.split_documents(data)
  return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
def download_hugging_face_embedding():
  emdeddings = HuggingFaceEmbeddings(model_name="sentence_transformers/all-MiniLM-L6-v2")
  return emdeddings

In [ ]:
embeddings = download_hugging_face_embedding()

In [ ]:
query_result = embeddings.embed_query("Hello All")
print("Length", len(query_result))

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGrpc as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = medbot

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

print(response)

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(documents=text_chunks,
                                               embeddings=embeddings,
                                               index_name=index_name)

In [ ]:
#embed each chunk and upsert the embedding into pinecone index
docsearch = PineconeVectorStore.from_existing_index(embeddings=embeddings,
                                                    index_name=index_name)

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k": 3})

In [ ]:
from langchain_groq import Groq

llm = Groq(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core import ChatPromptTemplate

system_prompt = (
"You are an assistant for question-answering tasks."
"Use the following pieces of retrieved context to answer"
"the question. If you don't know the answer, say that you "
"don't know. Use three sentences maximum and keep the "
"answer concise."
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages

  [

    ("system", system_prompt),

    ("human", "{input}"),

  ]

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>